In [1]:
# basic imports
import pandas as pd

# google imports 
import pandas_gbq 
from google.oauth2 import service_account 
from google.cloud import bigquery

In [2]:
credentials = service_account.Credentials.from_service_account_file(filename='gbq_keys.json',
                                                                    scopes=['https://www.googleapis.com/auth/cloud-platform'])

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "leandro-project"



In [3]:
# criando um objeto chamado client, que sera usado para chamar os metodos
client = bigquery.Client(credentials=credentials)

In [4]:
# construindo uma referencia apartir do dataset 'chicago_crime' que pertence ao projeto 'bigquery-public-data'
dataset_ref = client.dataset("br_poder360_pesquisas", project="basedosdados")

# criando um dataset
dataset = client.get_dataset(dataset_ref)

print('{}\n{}\n'.format(dataset_ref, dataset))

DatasetReference('basedosdados', 'br_poder360_pesquisas')
Dataset(DatasetReference('basedosdados', 'br_poder360_pesquisas'))



In [5]:
# listando tabelas
tables = list(client.list_tables(dataset))

# printando o nome das tabelas
for table in tables:  
    print(table.table_id)

microdados


In [6]:
table_ref = dataset_ref.table("microdados")
table = client.get_table(table_ref)


table.schema

[SchemaField('id_pesquisa', 'STRING', 'NULLABLE', 'ID da pesquisa', (), None),
 SchemaField('ano', 'INTEGER', 'NULLABLE', 'Ano', (), None),
 SchemaField('sigla_uf', 'STRING', 'NULLABLE', 'Sigla da Unidade da Federação', (), None),
 SchemaField('nome_municipio', 'STRING', 'NULLABLE', 'Nome do município', (), None),
 SchemaField('cargo', 'STRING', 'NULLABLE', 'Cargo', (), None),
 SchemaField('data', 'DATE', 'NULLABLE', 'Data', (), None),
 SchemaField('data_referencia', 'STRING', 'NULLABLE', 'Data de referência', (), None),
 SchemaField('instituto', 'STRING', 'NULLABLE', 'Instituto realizador', (), None),
 SchemaField('contratante', 'STRING', 'NULLABLE', 'Contratante', (), None),
 SchemaField('orgao_registro', 'STRING', 'NULLABLE', 'Orgão de registro', (), None),
 SchemaField('numero_registro', 'STRING', 'NULLABLE', 'Número de registro', (), None),
 SchemaField('quantidade_entrevistas', 'FLOAT', 'NULLABLE', 'Quantidade de entrevistas', (), None),
 SchemaField('margem_mais', 'FLOAT', 'NULL

# SELECT, DISTINCT, WHERE e LIMIT

In [7]:
# Query to select countries with units of "ppm"
query =  """
              SELECT *
              FROM `basedosdados.br_poder360_pesquisas.microdados`
              LIMIT 100
              """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()

# View top few rows of results
results.head()


,id_pesquisa,ano,sigla_uf,nome_municipio,cargo,data,data_referencia,instituto,contratante,orgao_registro,...,tipo,turno,tipo_voto,id_cenario,descricao_cenario,id_candidato_poder360,nome_candidato,sigla_partido,condicao,percentual
0,8023,2006,None,None,presidente,2006-09-19,18-19.set.06,Datafolha,None,None,...,estimulada,1,votos totais,20265,cenário único - estimulado,645,Heloísa Helena,Psol,0,9.0
1,8023,2006,None,None,presidente,2006-09-19,18-19.set.06,Datafolha,None,None,...,estimulada,1,votos totais,20265,cenário único - estimulado,327,Cristovam Buarque,PDT,0,2.0
2,8023,2006,None,None,presidente,2006-09-19,18-19.set.06,Datafolha,None,None,...,estimulada,1,votos totais,20265,cenário único - estimulado,1508,Rui Costa Pimenta,PCO,0,0.0
3,8023,2006,None,None,presidente,2006-09-19,18-19.set.06,Datafolha,None,None,...,estimulada,1,votos totais,20265,cenário único - estimulado,806,Eymael,PSDC,0,0.0
4,8023,2006,None,None,presidente,2006-09-19,18-19.set.06,Datafolha,None,None,...,estimulada,1,votos totais,20265,cenário único - estimulado,97,Ana Maria Rangel,PRP,0,0.0


Selecionar todos os intitutos de pesquisa apartir de 2006,

In [14]:
# Query to select countries with units of "ppm"
query =  """
              SELECT DISTINCT(instituto)
              FROM `basedosdados.br_poder360_pesquisas.microdados`
              WHERE ano >= 2006 
              LIMIT 100
              """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
results

,instituto
0,Datafolha
1,Vox Populi
2,IBPAD
3,Ibope
4,Malujoa Comunicações
...,...
95,Projetar
96,Access
97,FLS
98,MT Dados


Selecionar todos os candidatos a presidente no no de 2006, pelo instituto Datafolha,

In [11]:
# Query to select countries with units of "ppm"
query =  """
              SELECT DISTINCT(nome_candidato)
              FROM `basedosdados.br_poder360_pesquisas.microdados`
              WHERE ano = 2006 AND cargo = 'presidente' AND instituto = 'Datafolha'
              LIMIT 100
              """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
results

,nome_candidato
0,Heloísa Helena
1,Cristovam Buarque
2,Rui Costa Pimenta
3,Eymael
4,Ana Maria Rangel
5,indeciso / não sabe / não respondeu
6,Alckmin
7,Luciano Bivar
8,Lula
9,branco / nulo / indeciso / nenhum


In [12]:
# Query to select countries with units of "ppm"
query =  """
              SELECT DISTINCT(sigla_partido)
              FROM `basedosdados.br_poder360_pesquisas.microdados`
              WHERE ano >= 2014 
              AND sigla_partido != 'N/A' 
              AND sigla_partido != 'sem partido' 
              """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
results

,sigla_partido
0,MDB
1,Patriota
2,DC
3,PP
4,PMN
5,PC do B
6,PSB
7,PT
8,Psol
9,UP
